# Classification de genres musicaux

[GTZAN](http://opihi.cs.uvic.ca/sound/genres.tar.gz)

+ Description:
L'ensemble de données se compose de 1000 pistes audio d'une durée de 30 secondes. 

Il contient 10 genres, chacun représenté par 100 pistes. 

Les pistes sont toutes des fichiers audio mono 16 bits à 22050 Hz au format .wav.

Les genres sont :

+ blues
+ classique
+ country
+ disco
+ hip hop
+ jazz
+ metal
+ pop
+ reggae
+ rock

# Préparation des données

Le sample rate (taux d'échantillonage) correspond à la moyenne du nombre d'échantillons capturés en 1 seconde. 

Afin de constituer les données d'entrainements, chaque piste audio est découpée en piste de *n* secondes avec un pas de *m* secondes. 
(e.g., pour *n=2* et *m=1* : de 0 à 2 secondes, de 1 à 3 secondes ...).

Chaque piste audio de *n* secondes correspond à un array de taille [sample rate * *n*]

In [2]:
import os
import sys
sys.path.append("../")

import pandas as pd
pd.set_option('display.max_columns', 10000)

#Audio Processing Libraries
import librosa
from scipy import signal
#For Playing Audio
import IPython.display as ipd
#Array Processing
import numpy as np
#Data Visualization
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'pandas'

In [ ]:
from glob import glob

def prepare_tensor(data_dir, num_of_samples=44100, num_of_common=22050):
    full_data = []
    full_labels = []
    genres = os.listdir(data_dir)
    for i, genre in enumerate(genres):
        files = glob(os.path.join(data_dir, genre, '**'))
        for file in files:
            tmp_data = []
            data, _ = librosa.load(file, sr=num_of_common)
            #for offset in range(0, len(data), num_of_common):
            for offset in range(0, num_of_common*100, num_of_common):
                start = offset
                end   = offset + num_of_samples
                chunk = data[start:end]
                if(len(chunk)==num_of_samples):
                    tmp_data.append(chunk)
            tmp_labels = [i for _ in range(len(tmp_data))]
            full_labels += tmp_labels
            full_data += tmp_data
    return np.array(full_data), np.array(full_labels)


In [ ]:
DATA_DIR = '../data/01/genres/'

audio, labels = prepare_tensor(DATA_DIR)
print(audio.shape, len(labels))


In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(
    audio, labels,
    stratify=labels, test_size=0.2,
    random_state=777, shuffle=True
)

NameError: name 'audio' is not defined

In [ ]:
del audio, labels

In [ ]:
x_tr_features = x_tr.reshape(len(x_tr), -1, 1)
x_val_features = x_val.reshape(len(x_val), -1, 1)
print("Reshaped Array Size", x_tr_features.shape)

Reshaped Array Size (8000, 44100, 1)


In [ ]:
del x_tr, x_val

## simple CNN

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.metrics import CategoricalAccuracy
from keras.losses import CategoricalCrossentropy
from keras import backend as K

def cnn(x_tr):
  K.clear_session()
  inputs = Input(shape=(x_tr.shape[1], x_tr.shape[2]))
  #First Conv1D layer
  conv = Conv1D(8, 3, padding='same', activation='sigmoid')(inputs)
  conv = Dropout(0.3)(conv)
  conv = MaxPooling1D(10)(conv)
  #Second Conv1D layer
  conv = Conv1D(16, 3, padding='same', activation='sigmoid')(conv)
  conv = Dropout(0.3)(conv)
  conv = MaxPooling1D(10)(conv)
  #MaxPooling 1D
  conv = GlobalMaxPool1D()(conv)
  #Dense Layer
  conv = Dense(16, activation='sigmoid')(conv)
  outputs = Dense(10, activation='softmax')(conv)
  model = Model(inputs, outputs)
  model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer='adam',
    metrics=[
      "sparse_categorical_crossentropy",
      "sparse_categorical_accuracy"]
  )
  return model


model = cnn(x_tr_features)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 44100, 1)]        0         
                                                                 
 conv1d (Conv1D)             (None, 44100, 8)          112       
                                                                 
 dropout (Dropout)           (None, 44100, 8)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4410, 8)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4410, 16)          1424      
                                                                 
 dropout_1 (Dropout)         (None, 4410, 16)          0         
                                                             

In [ ]:
history = model.fit(
    x_tr_features, 
    y_tr, 
    epochs=10, 
    callbacks=[
        EarlyStopping(
            monitor='val_loss', 
            min_delta=.01, 
            patience=3, 
            restore_best_weights=True)
    ],
    batch_size=32, 
    validation_data=(x_val_features, y_val)
)


Epoch 1/10


2022-02-15 11:17:10.214504: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1411200000 exceeds 10% of free system memory.


800/800 [==============================] - 66s 82ms/step - loss: 2.2909 - sparse_categorical_crossentropy: 2.2909 - sparse_categorical_accuracy: 0.1244 - val_loss: 2.2601 - val_sparse_categorical_crossentropy: 2.2601 - val_sparse_categorical_accuracy: 0.1870
Epoch 2/10
800/800 [==============================] - 65s 82ms/step - loss: 2.1144 - sparse_categorical_crossentropy: 2.1144 - sparse_categorical_accuracy: 0.2171 - val_loss: 2.0753 - val_sparse_categorical_crossentropy: 2.0753 - val_sparse_categorical_accuracy: 0.2140
Epoch 3/10
800/800 [==============================] - 68s 85ms/step - loss: 2.0062 - sparse_categorical_crossentropy: 2.0062 - sparse_categorical_accuracy: 0.2477 - val_loss: 1.9953 - val_sparse_categorical_crossentropy: 1.9953 - val_sparse_categorical_accuracy: 0.2420
Epoch 4/10
800/800 [==============================] - 70s 87ms/step - loss: 1.9554 - sparse_categorical_crossentropy: 1.9554 - sparse_categorical_accuracy: 0.2828 - val_loss: 1.9446 - val_sparse_catego

KeyboardInterrupt: 

In [ ]:
model.load_weights('best_model.hdf5')


plt.plot(history.history['loss'])

plt.plot(history.history['val_loss'])

plt.title('Model loss')

plt.xlabel('Time')

plt.ylabel('epoch')

plt.legend(['train', 'validation'], loc='upper left')

plt.show()


NameError: name 'model' is not defined